In [ ]:
import numpy as np

In [ ]:
image = np.load("image.npy")
coordinates_img_lat = np.load("coordinates_img_x.npy")
coordinates_img_lon = np.load("coordinates_img_y.npy")

In [ ]:
image.shape, coordinates_img_lat.shape, coordinates_img_lon.shape

In [ ]:
coordinates_img_lat, coordinates_img_lon = coordinates_img_lat.round(4), coordinates_img_lon.round(4)

In [ ]:
value = coordinates_img_lon[0][64][64]
absolute_val_array = np.abs(coordinates_img_lon - value)
smallest_difference_index = np.unravel_index(absolute_val_array.argmin(), absolute_val_array.shape)
smallest_difference_index

In [ ]:
absolute_val_array.shape, absolute_val_array.shape

In [ ]:
(absolute_val_array + absolute_val_array).shape

In [ ]:
closest_element = coordinates_img_lon[smallest_difference_index]
closest_element

In [ ]:
center_dim, center_x, center_y = np.where(coordinates_img_lon == coordinates_img_lon[0][64][64])
center_dim, center_x, center_y

In [ ]:
center_dim, center_x, center_y = np.where(coordinates_img_lat == coordinates_img_lat[0][64][64])
center_dim, center_x, center_y

In [ ]:
center_dim, center_x, center_y = np.where((coordinates_img_lat == coordinates_img_lat[0][64][64]) & (coordinates_img_lon == coordinates_img_lon[0][64][64]))
center_dim, center_x, center_y = int(center_dim[0]), int(center_x[0]), int(center_y[0])
center_dim, center_x, center_y

In [ ]:
(int(center_lat-16)

In [ ]:
coordinates_img_lat[:, center_y-16:center_y+16, center_x-16:center_x+16].shape

In [ ]:
import csv

In [ ]:
station_ids = []
lats = []
lons = []
elevs = []
geohashs = []
stations = []

with open('ET/stations.csv', encoding='windows-1252', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        print(row["STATION_ID"], row["LAT"], row["LON"], row["ELEV"], row["GEOHASH"])
        station_ids.append(row["STATION_ID"])
        lats.append(float(row["LAT"]))
        lons.append(float(row["LON"]))
        elevs.append(int(row["ELEV"][:-3]))
        geohashs.append(row["GEOHASH"])
        stations.append((row["STATION_ID"], float(row["LAT"]), float(row["LON"]), int(row["ELEV"][:-3]), row["GEOHASH"]))
        

In [ ]:
stations[0][1]

## Cloud Coverage and Missing Pixel Coverage

In [ ]:
def compute_cloud_coverage(image):

    # compute width and height
    width = image.shape[2]
    height = image.shape[3]
    array = image[0, 10, :, :]

    # compute ratio of filled and no pixels
    cloud = 0
    clear_pixels = 0
    for i in range(0, height):
        for j in range(0, width):
            # remove and pixels ice/snow
             if (2800 <= array[i][j] < 3744) or (7840 > array[i][j] > 3788) or array[i][j] >= 7872 :
#             if ((2800 <= array[i][j] < 2976) or (6896 <= array[i][j] < 7072)) :
                cloud += 1
            else:
                clear_pixels += 1

    return cloud / (clear_pixels + cloud)

In [ ]:
def compute_pixel_coverage(image):
    
    # compute width and height
    width = image.shape[2]
    height = image.shape[3]
    array = image[0, 10, :, :]

    # compute ratio of filled and no pixels
    no_pixels = 0
    filled_pixels = 0
    for i in range(0, height):
        for j in range(0, width):
            if array[i][j] == 0 or array[i][j] == 1:
                no_pixels += 1
            else:
                filled_pixels += 1

    return filled_pixels / (filled_pixels + no_pixels)

## Combine Landsat and ET Datasets

In [ ]:
import glob
import csv
import numpy as np
from matplotlib.pyplot import imshow
from torchvision import transforms, utils
import os

In [ ]:
os.makedirs("Joined_Data_16x16", exist_ok = True)

In [ ]:
landsat_list = glob.glob("landsat16x16/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
counter = 0
for image in landsat_list:
    counter += 1
    print("Count: " + str(counter) + "/" + str(len(landsat_list)))
    data = np.load(image)
    with open('Stations\\' + image.split("\\")[1].split("_")[0] + '.csv', encoding='windows-1252', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            if ((row[0][-4:] + "_" + row[0][:-5] in image)):
                if ((float(row[1]) != -9999) and (float(row[1]) != -99999)):
                    np.save("Joined_Data_16x16\\" + image.split("\\")[1].replace(".npy", "_" + str(row[1]) + ".npy"), data)

In [ ]:
landsat_list = glob.glob("ET/Joined_Data_16x16/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
landsat_list = glob.glob("ET/Landsat16x16/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
for image in landsat_list:
    data = np.load(image)
    with open('ET/Stations\\' + image[16:22] + '.csv', encoding='windows-1252', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            if (image[16:22] in image and (row[0][-4:] + "_" + row[0][:-5] in image)):
                if ((float(row[1]) != -9999) and (float(row[1]) != -99999)):
                    if ((compute_pixel_coverage(data) == 1.0)):
                        if ((compute_cloud_coverage(data) <= 0.01)):
                            np.save("ET/Joined_Data_16x16_No_Cloud/" + image[16:-4] + "_" + str(row[1]) + ".npy", data)

In [ ]:
landsat_list = glob.glob("ET/Joined_Data_16x16_No_Cloud/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
landsat_list = glob.glob("ET/Landsat8x8/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
counter = 0
for image in landsat_list:
    data = np.load(image)
    with open('ET/Stations\\' + image[14:20] + '.csv', encoding='windows-1252', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            if (image[14:20] in image and (row[0][-4:] + "_" + row[0][:-5] in image)):
                if ((float(row[1]) != -9999) and (float(row[1]) != -99999)):
                    if ((compute_pixel_coverage(data) == 1.0)):
                        if ((compute_cloud_coverage(data) <= 0.01)):
                            np.save("ET/Joined_Data_8x8_No_Cloud/" + image[14:-4] + "_" + str(row[1]) + ".npy", data)

In [ ]:
landsat_list = glob.glob("ET/Joined_Data_8x8_No_Cloud/*.npy")

In [ ]:
len(landsat_list)

In [ ]:
landsat_list[0]

In [ ]:
temp = np.load(landsat_list[0])

In [ ]:
temp.shape

In [ ]:
landsat_list[0][11:17]

In [ ]:
landsat_list[0][11:-4]

In [ ]:
stations_list = glob.glob("ET/Stations/*")

In [ ]:
stations_list

In [ ]:
for image in landsat_list:
    data = np.load(image)
#     print(data.shape) 
#     print(image)
#     print('ET/Stations/' + image[11:17] + '.csv')
    with open('ET/Stations\\' + image[16:22] + '.csv', encoding='windows-1252', newline='') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader)
        for row in csvreader:
            if (image[16:22] in image and (row[0][-4:] + "_" + row[0][:-5] in image)):
                if ((float(row[1]) != -9999) and (float(row[1]) != -99999)):
                    np.save("ET/Joined_Data_16x16/" + image[16:-4] + "_" + str(row[1]) + ".npy", data)

In [ ]:
with open('ET/Stations\\' + "US-MOz" + '.csv', encoding='windows-1252', newline='') as csvfile:
    csvreader = csv.reader(csvfile)
    print(csvreader)
    next(csvreader)
    print(csvfile)
    for row in csvreader:
        print(row)
        print(row[0][-4:] + "_" + row[0][:-5])
        print(row[1])
        

In [ ]:
with open('ET/stations.csv', encoding='windows-1252', newline='') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        print(row["STATION_ID"], row["LAT"], row["LON"], row["ELEV"], row["GEOHASH"])
        station_ids.append(row["STATION_ID"])
        lats.append(float(row["LAT"]))
        lons.append(float(row["LON"]))
        elevs.append(int(row["ELEV"][:-3]))
        geohashs.append(row["GEOHASH"])
        stations.append((row["STATION_ID"], float(row["LAT"]), float(row["LON"]), int(row["ELEV"][:-3]), row["GEOHASH"]))

In [ ]:
import numpy as np

In [ ]:
here = np.random.rand(1, 11, 32, 32)
here2 = np.random.rand(1, 1, 32, 32)
here3 = np.random.rand(1, 1, 32, 32)

In [ ]:
np.concatenate([here, here2, here3], axis=1).shape

In [ ]:
import numpy as np

In [ ]:
temp1 = np.random.randn(1, 11, 32, 32)

In [ ]:
temp2 = np.random.randn(1, 32, 32)

In [ ]:
temp3 = np.concatenate((temp1, temp2.reshape(1, 1, 32, 32), temp2.reshape(1, 1, 32, 32)), axis=1)

In [ ]:
temp3.shape

# SSEBOP

In [1]:
import xarray as xr
from distributed_ssebop import ssebop
import glob
import numpy as np
import os 

In [2]:
os.makedirs("SSEBop_Results_0.8", exist_ok = True)

In [7]:
landsat_list = glob.glob("Joined_Data_16x16_Cloudy/*.npy")

In [8]:
len(landsat_list)

3555

In [11]:
counter = 0
for file in landsat_list:
    counter += 1
    print("Counter: " + str(counter) + "/" + str(len(landsat_list)))
    landsat_data = np.load(file)
    date = file.split("_")[-4] + "-" + file.split("_")[-3] + "-" + file.split("_")[-2]
    band3 = xr.DataArray(
        data=landsat_data[0, 2],
        dims=["y", "x"],
        coords=dict(
            lon=(["y", "x"], landsat_data[0,12]),
            lat=(["y", "x"], landsat_data[0,11]),
        ),
    )
    band4 = xr.DataArray(
        data=landsat_data[0, 3],
        dims=["y", "x"],
        coords=dict(
            lon=(["y", "x"], landsat_data[0,12]),
            lat=(["y", "x"], landsat_data[0,11]),
        ),
    )
    band8 = xr.DataArray(
        data=landsat_data[0, 7],
        dims=["y", "x"],
        coords=dict(
            lon=(["y", "x"], landsat_data[0,12]),
            lat=(["y", "x"], landsat_data[0,11]),
        ),
    )
    
    ssebop_img = ssebop.Ssebop(
        band3,
        band4,
        band8,
        date,
        et_reference_factor=0.8,
        et_reference_resample="linear",
        tmax_resample="linear",
        elev_resample="linear",
    )
    np.save(file.replace("Joined_Data_16x16_Cloudy", "SSEBop_Results_0.8"), np.concatenate((landsat_data[0], ssebop_img.et.data.reshape(1, 16, 16)), axis=0).reshape(1, 14, 16, 16))

Counter: 1/3555


D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'potential_evapotranspiration' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


Counter: 2/3555


D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'potential_evapotranspiration' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


Counter: 3/3555


...

D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'potential_evapotranspiration' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


Counter: 3554/3555


D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'potential_evapotranspiration' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


Counter: 3555/3555


D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'air_temperature' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(
D:\Evapo\Cloud_Removal\xarray\conventions.py:492: SerializationWarning: variable 'potential_evapotranspiration' has _Unsigned attribute but is not of integer type. Ignoring attribute.
  new_vars[k] = decode_cf_variable(


In [8]:
ssebop_img.et.data.shape

(16, 16)

In [15]:
landsat_data[0]

array([[[8216.    , 8260.    , 8276.    , ..., 8169.    , 8163.    ,
         8246.    ],
        [8256.    , 8339.    , 8316.    , ..., 8185.    , 8162.    ,
         8173.    ],
        [8270.    , 8410.    , 8364.    , ..., 8192.    , 8192.    ,
         8200.    ],
        ...,
        [8216.    , 8348.    , 8410.    , ..., 8058.    , 8102.    ,
         8133.    ],
        [8241.    , 8394.    , 8223.    , ..., 8065.    , 8146.    ,
         8305.    ],
        [8272.    , 8324.    , 8208.    , ..., 8039.    , 8118.    ,
         8203.    ]],

       [[7597.    , 7645.    , 7629.    , ..., 7550.    , 7512.    ,
         7584.    ],
        [7606.    , 7741.    , 7713.    , ..., 7521.    , 7467.    ,
         7449.    ],
        [7601.    , 7850.    , 7780.    , ..., 7499.    , 7515.    ,
         7527.    ],
        ...,
        [7576.    , 7790.    , 7820.    , ..., 7336.    , 7381.    ,
         7516.    ],
        [7587.    , 7774.    , 7591.    , ..., 7344.    , 7485.    ,
   